* 색인 & 가중치 => 학습
1. 수집한 문서 목록 가져오기
2. 각 문서에서 색인어 목록 추출 전처리 -> 어절 -> 형태소+명사+바이그램
3. 가중치 계산(TF-IDF) -> TDM -> TWM(with DocumentVectorLength)

* 질의 -> 색인과정의 2 to 3
4. 질의에서 색인어 추출
5. 가중치 계산
6. 유사도 계산(코싸인)
7. 유사도 순 정렬(거리-오름차순, 각도-내림차순)
8. 검색 결과 출력

In [3]:
def getFileList(base = "./", ext = "txt"): # 아무것도 안했다면 base는 현재 경로
    fileList = list()
    for file in listdir(base):
        if file.split(".")[-1] == ext: # .을 기준으로 split한 것이 txt인지 검사
            fileList.append("{0}/{1}".format(base, file))
            
    return fileList

# txt 컨텐츠를 편하게 읽어오기 위한 함수
def getContent(file): 
    with open(file, encoding="UTF-8") as f:
        content = f.read()
    return content

In [4]:
def ngramEojeol(sentence, n=2): # sentence를 받아 어절 단위로 분리해주는 함수
    '''
    입력:     단어1,   단어2,   단어3,  단어4 : 4
    출력(2) : 단어12,  단어23,  단어34 :        3 - n + 1
    출력(3) : 단어123, 단어234         :        2 - n + 1
    '''
    tokens = sentence.split()
    ngram = []
    
    for i in range(len(tokens) - n + 1):
        ngram.append(' '.join(tokens[i:i + n]))    
        
    return ngram

In [5]:
def ngramUmjeol(term, n = 2): # 음절 단위로 구분하는 함수. sentence를 받아 2개(n=2)씩 쪼갠다.

    ngram = []
    
    for i in range(len(term) - n + 1):
        # ngram.append(tokens_ngram[i:i+n]) # 방법1
        # ngram.append(tuple(tokens_ngram[i:i+n])) # 방법2 (튜플로 반환 시 키값을 쓸 수 있음)
        ngram.append(''.join(term[i:i + n])) # 방법3
        
    return ngram

In [6]:
import re
from string import punctuation


def getPatternList():
    patternList = dict()
    pattern = re.compile(r"[%s]{1,}" % re.escape(punctuation)) # punctuation 안의 특수문자가 두번이상 반복되는 모든 문자에 대해 패턴 정의
    patternList["Punctuation"] = pattern
    pattern2 = re.compile(r"([A-Za-z0-9\-\_\.]{3,}@[A-Za-z0-9\-\_]{3,}(.[A-Za-z]{2,})+)") #이메일주소제거패턴
    patternList["Email"] = pattern2
    pattern3 = re.compile(r"([A-Za-z0-9\-\_]{1,}(.[A-Za-z]{2,})+)") # 신문사도메인패턴
    patternList["Domain"] = pattern3
    pattern4 = re.compile(r"\s{2,}") # 공백제거
    patternList["Whitespace"] = pattern4
    pattern5 = re.compile(r"([^ㄱ-ㅎㅏ-ㅣ가-힣0-9]+)") # 한글이 아닌 영어 기호 제거
    patternList["Korean"] = pattern5
    pattern6 = re.compile(r'[A-Za-z-_]{8,}') # 영어 (대소문자) +\-\_ 제거 (8글자 이상)
    patternList["ElimLongEng"] = pattern6
    pattern7 = re.compile(r"를 2019 및 금지|네이버 채널에서|저작권자 주 블로터앤미디어 저작권자를 명기하고 내용을 변경하지 않으며 비상업적으로 이용하는 조건아래 가능합니다|"
                          r"디지털 마케팅의 미래 아이포럼|무단전재|오류를 우회하기 위한 함수 추가|재배포|기자|아이뉴스24|구독해주세요|"
                          r"고수들의 재테크 비법|이건희칼럼|성공을 꿈꾸는 사람들의 경제 뉴스|머니 및 금지|및 금지|"
                          r"네이버 모바일에서|네이버에서 헤럴드경제|채널 구독하기|헤럴드 리얼라이프 헤럴드경제 사이트 바로가기 헤럴드경제 무단 전재 및 금지|"
                          r"전자신문|바로가기|전자신문인터넷|인터넷 및 금지|중앙일보|친구추가|구독 1위|"
                          r"한국경제|재배포|기사제보|보도자료|한경닷컴|네이버에서|자세히보기|자세히 보기|모바일한경|구독신청|"
                          r"머니투데이|돈이 보이는 리얼타임 뉴스|연예뉴스|파이낸셜뉴스|무단 전재 금지") # 높은 가중치를 갖는 단어를 하드코딩 데이터로 제거
    patternList["ElimRecWord"] = pattern7
    
    return patternList

In [44]:
from os import listdir

FileList = getFileList("C:/Users/brsta/ICT_AI_AdvanceClass_NLP/0314_DownloadedNewstxts/")
NewsContent = list() 

for i in range(len(FileList)):
    NewsContent.append(getContent(FileList[i])) # len(NewsContent) = 180


In [47]:
patternList = getPatternList()
for _ in ["Korean", "Whitespace", "Punctuation", "ElimLongEng", "Email", "Domain", "ElimRecWord"]: 
    for i in range(len(FileList)):
        NewsContent[i] = patternList[_].sub(" ", NewsContent[i]) 
print(NewsContent[0])


 오히려 단체측 무선이어폰 연구는 존재치도 않아 국내 참여 연구자도 어리둥절 이엠에프사이언티스에서 일부 언론 보도를 부정하며 직접 보내온 이메일 내용이다 이엠에프사이언티스트 관계자인 조엘 모스코위츠 미국 버클리 캘리포니아대 가정사회건강연구소 소장은 무선 이어폰의 건강 유해성에 대한 보도를 부정했다 고재원 1212 18일 오전 전세계 과학자들이 애플 에어팟과 같은 무선 이어폰이 암 발생 위험을 키울 수 있다는 호소문을 국제연합 과 세계보건기구 에 제출했다는 일부 국내외 언론의 보도가 나오면서 불안감이 확산되고 있다 하지만 실제 이 단체와 호소문에 이름을 올린 과학자들에게 확인한 결과 호소문은 4년전 제출됐던 것이며 또 특정 제품이나 제조사를 언급하지 않은 것으로 확인됐다 18일 데일리메일과 등 국내외 일부 언론에 따르면 전 세계 과학자 247명이 무선 이어폰의 비이온화 전자기장 이 암을 유발할 위험 우려가 있다며 과 에 호소문을 제출했다고 전했다 이들 매체들은 호소문에 애플 에어팟이 에 관한 법적 기준치를 준수하고 있지만 장시간 노출될 경우 건강에 좋지 않을 수 있다 는 내용이 포함됐다고 보도했다 이들 매체들은 이 호소문에는 전 세계 42개국 과학자 247명이 서명을 했다고 전했다 여기에는 한국의 연세대 한양대 가톨릭대 단국대 중앙대 경북대 한림대 소속 과학자 15명의 이름도 포함됐다 하지만 취재 결과 호소문 작성을 주도한 비영리단체 이엠에프사이언티스트 는 애플 에어팟과 같은 무선 이어폰에 대한 유해성을 주장하지 않은 것으로 확인됐다 동아사이언스가 이엠에프사이언티스트에 직접 이메일로 확인한 결과 일부 언론 보도가 호소문에 대한 부정확한 내용을 담고 있다 며 무선 블루투스의 자기장에 머리가 장시간 노출될 시의 안정성에 대한 연구는 존재하지 않는다 고 밝혔다 이엠에프사이언티스트에 따르면 이 단체는 지난 2015년 5월 실제로 전세계 과학자 190명의 서명을 받아 과 유엔환경계획 에 국제 과학자 호소문 을 제출하기는 했다 당시 호소문에는 전세계 저명한 학술지에 게재된

In [46]:
from nltk.tokenize import sent_tokenize, word_tokenize
from konlpy.tag import Kkma
ma = Kkma()
# len(sent_tokenize(NewsContent[0])) # 구두점이 사라져서 토큰화 못함.

dictTerm = list()
dictPos = list()
dictNoun = list()
dictNgram = list()
th=1

for sentence in sent_tokenize(NewsContent[0]):
    for token in word_tokenize(sentence):
        if len(token) > th:
            dictTerm.append(token)
            # 아래서부터는 list이기때문에 extend를 사용해야 함. 
            dictPos.extend([morpheme for morpheme in ma.morphs(token) if len(morpheme) > th]) # 형태소 분석결과를 extend
            dictNoun.extend([noun for noun in ma.nouns(token) if len(noun) > th]) # 명사 단위로 잘라 extend
            dictNgram.extend(ngramUmjeol(token)) # 바이그램을 리턴
            
# 빠른 속도 및 중복 데이터 처리를 위해 set사용
dictTerm = list(set(dictTerm))
dictPos = list(set(dictPos))
dictNoun = list(set(dictNoun))
dictNgram = list(set(dictNgram))

In [31]:
len(dictTerm), len(dictPos), len(dictNoun), len(dictNgram)


754

In [43]:
print(dictPos)

['이번', '장시간', '장과', '재원', '190', '교수', '영향', '기구', '대하', '제조사', '못하', '특정', '세계', '과학자', '나오', '비영리', '이미', '유엔', '신용', '기억', '실제로', '법적', '경북', '매체', '30', '하다', '호소문', '성에', '사이', '결과', '방지', '는데', '확산', '애플', '이어폰', '지방', '라며', '가져가', '에게', '부터', '여부', '연구소', '당시', '이나', '국내', '학술지', '늘어나', '언론', '준수', '사회', '계획', '발표', '어리둥절', '마이크로', '여기', '한국', '취재', '데일리', '5000', '검증', '주장', '주도', '다른', '한림', '자인', '하지만', '문제', '발생', '대략', '유해', '동의', '안정성', '오전', '유해성', '통하', '근거', '기준치', '연구자', '이메일', '지만', '관계자', '일부', '불안감', '전하', '주제', '에어팟', '위험', '보도', '올리', '지속적', '스트', '소속', '환경', '명단', '강대', '말하', '에서', '필요', '전세계', 'ㄴ다고', '개국', '가까이', '전자기', '주변', '참여', '미국', '이름', '따르', '내리', '현재', '생화', '최근', '티스', '사이언스', '버클리', '가톨릭대', '우리', '부정확', '자체', '인체', '동안', '중앙대', '국내외', '새삼', '국립대', '가정', '단국', '노출', '모스코', '건강', '메일', '보내오', '수많', '단체', '캘리포니아', '블루투스', '전화', '연합', '제출', '15', '자기장', 'ㄴ다', '해당', '시의', '밝히', '새로', '연세대', '으며', '지나다니', '다고', '아니', '다며', '관여', '다는', '비이온', '화학', '18', '국제

In [33]:

len(list(set(dictTerm + dictPos + dictNoun + dictNgram)))


754

In [34]:
print(dictTerm)

['15명의', '장시간', '같은', '캘리포니아대', '서강대', '한림대', '제출됐으며', '불안감이', '확인한', '호소문은', '영향을', '언급하고', '나오면서', '있지', '확산되고', '특정', '세계', '과학자', '42개국', '이미', '무선이어폰', '존재치도', '언급하지', '단체는', '실제로', '법적', '과학자들의', '이덕환', '결과', '호소문', '이들', '포함됐다고', '있는데', '포함됐다', '전했다', '통신용', '내용이', '세계보건기구', '애플', '자기장에', '난다', '주변에', '2015년', '언론의', '제품이나', '건강에', '황당하다', '당시', '내용이다', '국내', '마이크로파가', '관계자인', '에어팟과', '따르면', '유발할', '우려가', '관한', '경북대', '게재된', '언론', '서명했다고', '동아사이언스가', '교수는', '이름도', '30년간', '올린', '호소문에는', '블루투스의', '아직까지', '어리둥절', '암을', '한양대', '취재', '인체에', '확실히', '소장은', '현재는', '다른', '좋지', '근거로', '않아', '주장하지', '필요하다는', '하지만', '제출됐던', '발생', '대략', '부정하며', '기준치를', '있지만', '연구를', '오전', '이메일로', '247명이', '서명했던', '유해성', '이엠에프사이언티스', '지방국립대', '된다는', '대한', '몰랐다', '단체측', '이메일', '지난', '않았고', '연구들이', '통해', '연구자도', '데일리메일과', '조엘', '부정했다', '동의하는', '일부', '190명의', '18일', '서명자인', '위험', '15명도', '않는다', '확인하지', '밝혔다', '키울', '전자기장', '아닌', '건강과', '소속', '247명으로', '담았다', '못했다고', '이엠에프사이언티스트', '명단이', '전세계', '부정확한', '늘어났다', '

In [9]:
collection = []
# print(type(FileList[0]))

for i in range(len(NewsContent)):
    collection.append((str(FileList[i]), NewsContent[i]))
print(type(collection[10][1]))

<class 'str'>


In [10]:
# in-memory (Hash Key 값)
# 전체 색인어 목록(Dictionary)
# {단어1:포스팅위치, 단어2:포스팅위치, ...}
globalLexicon = dict()

# 전체 문서 목록(Dictionary)
# [0:문서1, 1:문서2, ...]
globalDocument = list()

# disk
# 사전에 있는 색인어 중, 어느 문서에서, 몇 번 나타났는지
# [(단어 idx, 문서 idx, 빈도, 다음주소), ...]
# [0:Tuple(lexiconIdx, documentIdx, freq, 다음포스팅위치-fptr)]
# 메모리 X, File OK
globalPosting = list()


In [11]:
for (docName, docContent) in collection:
    # Pointer 대체용, Key, Document 이름은 절대로 겹치지 않는다는 가정. 
    docIdx = len(globalDocument)
    globalDocument.append(docName)
    
    # {단어idx:빈도, 단어idx:빈도, ...}
    localPosting = dict()
    
    # Local 작업
    for term in docContent.lower().split():
        # Local에 대해서 수행한 후 없으면 새 posting으로 추가
        if term not in localPosting.keys():
            localPosting[term] = 1 # dict
        # 있으면, 빈도 증가
        else:
            localPosting[term] += 1
     
    # Global Marge
    # fp -> struct(단어, 빈도) (localPosting)
    for indexTerm, termFreq in localPosting.items(): # indexTerm : str,termFreq : int 
        if indexTerm not in globalLexicon.keys(): 
            lexiconIdx = len(globalLexicon)
            postingIdx = len(globalPosting) # fseek
            postingData = (lexiconIdx, docIdx, termFreq, -1)
            globalPosting.append(postingData)
            globalLexicon[indexTerm] = postingIdx # globalPosting 위치(ptr:idx)
        else: # 기존 단어의 idx 가져오기
            lexiconIdx = list(globalLexicon.keys()).index(indexTerm)
            postingIdx = len(globalPosting)
            beforeIdx = globalLexicon[indexTerm]
            postingData = (lexiconIdx, docIdx, termFreq, beforeIdx)
            globalPosting.append(postingData)
            globalLexicon[indexTerm] = postingIdx
            
#     print(localPosting)
# print(globalDocument)

#         if term not in globalLexicon.keys():
#             lexiconIdx = len(globalLexicon) 0

In [23]:
# globalLexicon, globalDocument

In [22]:
# globalPosting

In [19]:
# 확인용
for indexTerm, postingIdx in globalLexicon.items():
    # indexTerm:단어: postingIdx:위치, ...
    # print(indexTerm)
    
    while True: # Posting Next:-1
        if postingIdx == -1:
            break
            
        postingData = globalPosting[postingIdx]
        # print('  DocName:{0} - TermFreq:{1} - Next:{2}'.format(globalDocument[postingData[1]], postingData[2], postingData[3]))
        postingIdx = postingData[3]

In [21]:
globalPosting[globalLexicon['출시가']]

(925, 25, 1, 4383)

In [67]:
from nltk.tokenize import sent_tokenize, word_tokenize
from konlpy.tag import Kkma
ma = Kkma()

dictTerm = list()
dictPos = list()
dictNoun = list()
dictNgram = list()
th = 1

for sentence in sent_tokenize(NewsContent[0]):
    for token in word_tokenize(sentence):
        if len(token) > th:
            dictTerm.append(token)
            # 아래서부터는 list이기때문에 extend를 사용해야 함. 
            dictPos.extend([morpheme for morpheme in ma.morphs(token) if len(morpheme) > th]) # 형태소 분석결과를 extend
            dictNoun.extend([noun for noun in ma.nouns(token) if len(noun) > th]) # 명사 단위로 잘라 extend
            dictNgram.extend(ngramUmjeol(token)) # 바이그램을 리턴
            
# 빠른 속도 및 중복 데이터 처리를 위해 set사용
dictTerm = list(set(dictTerm))
dictPos = list(set(dictPos))
dictNoun = list(set(dictNoun))
dictNgram = list(set(dictNgram))